In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.messages import HumanMessage
from langchain.agents.middleware import SummarizationMiddleware

load_dotenv(dotenv_path="../.env")

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always Rainy in {city}!"

model = ChatOpenAI(
    base_url=os.getenv("MODEL_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model=os.getenv("MODEL_NAME", "gpt-3.5-turbo"),
    temperature=0
)

In [6]:
# 降低token阈值来更容易触发总结
print("=== 创建低阈值版本的 SummarizationMiddleware ===")

# 创建一个更容易触发总结的agent
agent_low_threshold = create_agent(
    model=model,
    tools=[get_weather_for_location],
    middleware=[
        SummarizationMiddleware(
            model=model,
            max_tokens_before_summary=20,  # 降低到20 tokens
            messages_to_keep=2,  # 只保留3条消息
        ),
    ],
)

print("✅ 已创建低阈值版本的agent (20 tokens触发总结)")

# 测试低阈值版本
def test_low_threshold():
    """测试低阈值版本的 SummarizationMiddleware"""
    print("\n=== 测试低阈值版本 ===")
    
    long_messages = [
        "请详细告诉我上海的天气情况，并且根据天气情况给我列出出行指南。",
        "请简单告诉我三亚的天气情况。",
    ]
    
    for i, message in enumerate(long_messages):
        print(f"\n--- 长消息测试 {i+1} ---")
        try:
            response = agent_low_threshold.invoke({"messages": [HumanMessage(content=message)]})
            # 可以看到有在对model提问前进行了总结
            print(f"回复: {response}")
        except Exception as e:
            print(f"错误: {e}")
test_low_threshold()

=== 创建低阈值版本的 SummarizationMiddleware ===
✅ 已创建低阈值版本的agent (20 tokens触发总结)

=== 测试低阈值版本 ===

--- 长消息测试 1 ---
回复: {'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nUser requested detailed weather information for Shanghai and a travel guide based on the weather.', additional_kwargs={}, response_metadata={}, id='41dcf905-424c-4e0e-9a23-a7ba502b9584'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 519, 'prompt_tokens': 456, 'total_tokens': 975, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 490, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': '', 'id': '021760599650425f8c6a85cf57ff2d1ecf10d9f546f115817ec39', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--46

In [8]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

human_interrupt_agent = create_agent(
    model=model,
    tools=[get_weather_for_location],
    middleware=[
            HumanInTheLoopMiddleware(
            interrupt_on={
                # 人工判断是否支持调用获取本地天气
                "get_weather_for_location": {
                    "allowed_decisions": ["approve", "reject"],
                },
            }
        ),
    ],
)

messages = "请简要告诉我上海的天气情况。"

response = human_interrupt_agent.invoke({"messages": [HumanMessage(content=messages)]})

# 可以看到在call tools之前，已经中断，等待新的决策输入继续运行
print(response)

{'messages': [HumanMessage(content='请简要告诉我上海的天气情况。', additional_kwargs={}, response_metadata={}, id='50c1b511-34cd-4f29-9024-3e72ba269135'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 418, 'total_tokens': 497, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 46, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': '', 'id': '0217606024572550b5206b6c0f3bb2682a16c13f4ce95f3c73775', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--06bb4dc8-282b-4adb-8d83-a737d93cd42e-0', tool_calls=[{'name': 'get_weather_for_location', 'args': {'city': '上海'}, 'id': 'call_w43ukxmh2zaopj48fphjfsuh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 418, 'output_tokens': 79, 'total_tokens':

In [9]:
from langchain.agents.middleware import PIIMiddleware

redact_agent = create_agent(
    model=model,
    middleware=[
        # Redact emails in user input
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        # Mask credit cards (show last 4 digits)
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        # Custom PII type with regex
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",  # Raise error if detected
        ),
    ],
)

messages="我的邮箱是junbo.wang952@gmail.com，请问我的邮箱是什么？"
response = redact_agent.invoke({"messages": [HumanMessage(content=messages)]})
# 不应该输出正确的邮箱
print(response)

{'messages': [HumanMessage(content='我的邮箱是junbo[REDACTED_EMAIL]，请问我的邮箱是什么？', additional_kwargs={}, response_metadata={}, id='1f7bf9c7-a50d-4129-9a1e-f7cf965fe3c4'), AIMessage(content='你提供的邮箱信息中，`[REDACTED_EMAIL]` 是被隐私保护（或编辑）隐藏的部分，仅从你给出的内容能看到邮箱的前缀部分是 `junbo`，但完整的邮箱地址（比如后缀 `@xxx.com` 这类域名及中间可能的内容）因为被隐藏，无法从当前信息中获取到完整的邮箱。  \n\n如果你想确认完整邮箱，需要你回忆或补充被 `[REDACTED_EMAIL]` 隐藏的那部分内容（比如域名、子域名等），例如完整邮箱可能是类似 `junbo@xxx.com`（`xxx` 是你实际的域名）的形式，但具体需要你结合自己的注册信息来确定。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 399, 'prompt_tokens': 74, 'total_tokens': 473, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 240, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': '', 'id': '02176061308177632c68f27fb844bcdaeb8e6e5beffe87142a049', 'service_tier': None, 'finish_reason': 'sto